<h1>ECOSISTEMA SPARK</h1>

<h1>Práctica 2: SparkSQL</h1>

<h1>TAREA 2</h1>


<h4>Autor: Guillermo García López</h4>

<h1><font color='gray'>Objetivo 1<font></h1>

In [1]:
# Comando para habilitar scrolling horizontal en celdas
from IPython.core.display import HTML
display(HTML("<style>pre { white-space: pre !important; }</style>"))

In [2]:
# Se crea sparksession para utilidades SparkSQL
from pyspark.sql import SparkSession, Row

spark = SparkSession \
    .builder \
    .appName("Python Spark SQL - Practica 2") \
    .config("spark.some.config.option", "some-value") \
    .getOrCreate()
    
# Se crea SparkContext() para utilidades sobre RDD's
sc = SparkContext.getOrCreate()

Vamos a realizar la comparación entre tamaños de los formatos .csv y .parquet con el fichero 
'Phones_acceloremeter.csv'.

In [3]:
# Importamos librería os para acceder a la ruta del fichero
import os
os.path.getsize('./small_data/Phones_accelerometer.csv')

997515

Se cargan los ficheros .csv con la utilidad spark.read.load, y se guarda en formato .parquet, recopilando
los tamaños de ambos formatos:

In [4]:
# Chequeamos si el directorio con los ficheros .parquet de salida está vacío. Si no, borramos el contenido
# antes de guardar los ficheros parquet:
import shutil

if len(os.listdir('./output_parquet/')) == 0:
    pass
else:
    for file in os.listdir('./output_parquet/'):
        shutil.rmtree('./output_parquet/' + file + '/')

In [5]:
dfPhonesAcc = spark.read.load('./small_data/Phones_accelerometer.csv', format='csv')
dfPhonesAcc.write.save('./output_parquet/Phones_accelerometer.parquet', format='parquet')
phAcc_csv, phAcc_pqt = os.path.getsize('./small_data/Phones_accelerometer.csv'), \
                       os.path.getsize('./output_parquet/Phones_accelerometer.parquet')

In [6]:
dfPhonesGyr = spark.read.load('./small_data/Phones_gyroscope.csv', format='csv')
dfPhonesGyr.write.save('./output_parquet/Phones_gyroscope.parquet', format='parquet')
phGyr_csv, phGyr_pqt = os.path.getsize('./small_data/Phones_gyroscope.csv'), \
                       os.path.getsize('./output_parquet/Phones_gyroscope.parquet')

In [7]:
dfWatAcc = spark.read.load('./small_data/Watch_accelerometer.csv', format='csv')
dfWatAcc.write.save('./output_parquet/Watch_accelerometer.parquet', format='parquet')
WatAcc_csv, WatAcc_pqt = os.path.getsize('./small_data/Watch_accelerometer.csv'), \
                         os.path.getsize('./output_parquet/Watch_accelerometer.parquet')

In [8]:
dfWatGyr = spark.read.load('./small_data/Watch_gyroscope.csv', format='csv')
dfWatGyr.write.save('./output_parquet/Watch_gyroscope.parquet', format='parquet')
WatGyr_csv, WatGyr_pqt = os.path.getsize('./small_data/Watch_gyroscope.csv'), \
                         os.path.getsize('./output_parquet/Watch_gyroscope.parquet')

Creamos un rdd con la información de los ficheros .csv, ficheros .parquet y sus tamaños respectivos:

In [9]:
filesizes = [('Phones_accelerometer', phAcc_csv, phAcc_pqt), ('Phones_gyroscope', phGyr_csv, phGyr_pqt),
             ('Watch_accelerometer', WatAcc_csv, WatAcc_pqt), ('Watch_gyroscope', WatGyr_csv, WatGyr_pqt)]

In [10]:
rdd = sc.parallelize(filesizes)
aux = rdd.map(lambda el: Row(file=el[0], csvSize=int(el[1]), parquetSize=int(el[2]))) 
df = spark.createDataFrame(aux)

Y mostramos la tabla con los datos pedidos (tamaños en bytes):

In [11]:
df.select('file', 'csvSize', 'parquetSize').show()

+--------------------+-------+-----------+
|                file|csvSize|parquetSize|
+--------------------+-------+-----------+
|Phones_accelerometer| 997515|       4096|
|    Phones_gyroscope|1100646|       4096|
| Watch_accelerometer| 842763|       4096|
|     Watch_gyroscope| 894796|       4096|
+--------------------+-------+-----------+



Como vemos, los ficheros parquet ocupan muchísimo menos espacio en disco que los csv's.

<h1><font color='gray'>Objetivo 2<font></h1>

Se comparan las ejecuciones de los notebooks de la P1 (ejecución con RDD's), de la tarea 1 de esta misma
práctica (dataframes a partir de RDD's), así como de la realización con dataframes a partir de los ficheros parquet
y por último de la ejecución con dataframes a partir de los ficheros csv originales:

- Caso 1, P1 (RDD's):

In [12]:
#%run ../P1/P1_2_Spark_GuillermoGarciaLopez.ipynb
print('Tiempo de ejecución para rdd\'s: {0}'.format(3.7700133323669434))

Tiempo de ejecución para rdd's: 3.7700133323669434


<hr>


- Caso 2, P2 Tarea 1 (DataFrames a partir de RDD's) (en este caso no puede ejecutarse desde aquí, 
  apuntamos el tiempo que tarda):

In [13]:
#%run ./P2_Tarea1_Spark_GuillermoGarciaLopez.ipynb
print('Tiempo de ejecución para dataframes a partir de rdd\'s: {0}'.format(9.54147219657898))

Tiempo de ejecución para dataframes a partir de rdd's: 9.54147219657898


<hr>

- Caso 3: DataFrames a partir de ficheros parquet:

In [14]:
import time as t
init_time = t.time()

Cargamos los ficheros parquet en dataframes:

In [15]:
dfPhAccPqt  = spark.read.parquet('./output_parquet/Phones_accelerometer.parquet/')
dfPhGyrPqt  = spark.read.parquet('./output_parquet/Phones_gyroscope.parquet/')
dfWatAccPqt = spark.read.parquet('./output_parquet/Watch_accelerometer.parquet/')
dfWatGyrPqt = spark.read.parquet('./output_parquet/Watch_gyroscope.parquet/')

In [16]:
print(t.time() - init_time)

0.5779860019683838


Renombramos las columnas ya que parquet no las carga nombradas:

In [17]:
a = t.time()

In [18]:
'''
Función que selecciona las columnas del dataframe que nos interesan y las renombra, además de castearlas al
formato conveniente:
_c0 -> Index           (int)
_c1 -> Arrival_Time    (int)
_c2 -> Creation_Time   (int)
_c3 -> x               (float)
_c4 -> y               (float)
_c5 -> z               (float)
_c6 -> User            (string)
_c7 -> Model           (string)
_c8 -> Device          (string)
_c9 -> Gt              (string)

Input: dataframe
Output: dataframe
'''
def selectAndRename(inDf):
    
    outDf = inDf.select(inDf["_c6"], inDf["_c7"], inDf["_c9"], inDf["_c3"], inDf["_c4"], inDf["_c5"]) \
                .withColumnRenamed("_c6", "user") \
                .withColumnRenamed("_c7", "model") \
                .withColumnRenamed("_c9", "gt") \
                .withColumnRenamed("_c3", "x") \
                .withColumnRenamed("_c4", "y") \
                .withColumnRenamed("_c5", "z")
                
    outDf = outDf.selectExpr("cast(user as string) user", "cast(model as string) model", \
                             "cast(gt as string) gt", "cast(x as float) x", "cast(y as float) y", \
                             "cast(z as float) z")
    
    return outDf

In [19]:
dfPhAccPqt  = selectAndRename(dfPhAccPqt)
dfPhGyrPqt  = selectAndRename(dfPhGyrPqt)
dfWatAccPqt = selectAndRename(dfWatAccPqt)
dfWatGyrPqt = selectAndRename(dfWatGyrPqt)

In [20]:
print(t.time() - a)

0.6503894329071045


Y una vez creados los dataframe con el esquema adecuado, realizamos la misma operativa que en P2_Tarea1:

In [21]:
b = t.time()

In [22]:
phonesStats = dfPhAccPqt.join(dfPhGyrPqt, ['user', 'model', 'gt', 'x', 'y', 'z']  ,'full')
watchesStats = dfWatAccPqt.join(dfWatGyrPqt, ['user', 'model', 'gt', 'x', 'y', 'z']  ,'full')

In [23]:
Stats = phonesStats.unionAll(watchesStats)
Stats.createOrReplaceTempView('statsTable')

In [24]:
print(t.time()-b)

0.28332090377807617


In [25]:
query = "SELECT user, model ,gt ,mean(x), mean(y), mean(z) \
                ,std(x), std(y), std(z), max(x), max(y), max(z), min(x) \
                ,min(y), min(z) \
         FROM statsTable \
         GROUP BY user, model, gt"

In [26]:
c = t.time()

In [27]:
result = spark.sql(query)

In [28]:
print(t.time() - c)

5.475210666656494


In [29]:
#result.show()

In [30]:
#print(t.time() - c)

In [31]:
end_time = t.time()
print("Tiempo de ejecución dataframes desde ficheros parquet: {0}".format(end_time - init_time))

Tiempo de ejecución dataframes desde ficheros parquet: 9.311946392059326


<hr>

- Caso 4: DataFrames a partir de ficheros csv:

In [32]:
init_time = t.time()

Se crean los dataframes a partir de los csv's:

In [33]:
dfPhAccCsv  = spark.read.csv('./small_data/Phones_accelerometer.csv')
dfPhGyrCsv  = spark.read.csv('./small_data/Phones_gyroscope.csv')
dfWatAccCsv = spark.read.csv('./small_data/Watch_accelerometer.csv')
dfWatGyrCsv  = spark.read.csv('./small_data/Watch_gyroscope.csv')

Renombramos y casteamos las columnas del dataframe:

In [34]:
dfPhAccCsv  = selectAndRename(dfPhAccCsv)
dfPhGyrCsv  = selectAndRename(dfPhGyrCsv)
dfWatAccCsv = selectAndRename(dfWatAccCsv)
dfWatGyrCsv = selectAndRename(dfWatGyrCsv)

Y una vez creados los dataframe con el esquema adecuado, realizamos la misma operativa que en P2_Tarea1:

In [35]:
phonesStats = dfPhAccCsv.join(dfPhGyrCsv, ['user', 'model', 'gt', 'x', 'y', 'z']  ,'full')
watchesStats = dfWatAccCsv.join(dfWatGyrCsv, ['user', 'model', 'gt', 'x', 'y', 'z']  ,'full')

In [36]:
Stats = phonesStats.unionAll(watchesStats)
Stats.createOrReplaceTempView('statsTable')

Y se realiza la query sobre los datos:

In [37]:
result = spark.sql(query)
result.show()

+----+------+-----+-------------------+-------------------+-------------------+------------------------------+------------------------------+------------------------------+----------+------------+------------+-----------+-----------+----------+
|user| model|   gt|             avg(x)|             avg(y)|             avg(z)|stddev_samp(CAST(x AS DOUBLE))|stddev_samp(CAST(y AS DOUBLE))|stddev_samp(CAST(z AS DOUBLE))|    max(x)|      max(y)|      max(z)|     min(x)|     min(y)|    min(z)|
+----+------+-----+-------------------+-------------------+-------------------+------------------------------+------------------------------+------------------------------+----------+------------+------------+-----------+-----------+----------+
|   a|  gear| null| -4.625535975612993| -1.757118124303339|-0.5569131525539762|             4.644796095861487|            1.7196641033507496|           0.48540930881829963| 0.0561927|-0.018109497|-0.055127434| -9.3583355| -3.5942953|-1.1420342|
|   a|nexus4|stand| 

In [38]:
end_time = t.time()
print("Tiempo de ejecución dataframes desde ficheros csv: {0}".format(end_time - init_time))

Tiempo de ejecución dataframes desde ficheros csv: 7.144015789031982
